In [605]:
import numpy as np
from sympy import symbols, cos, sin, pi, simplify, sqrt, atan2, asin, acos
from sympy.matrices import Matrix
from numpy import array
from numpy.linalg import inv




In [607]:
d1, d2, d3, d4, d5, d6, d7 = symbols('d1:8')
a0, a1, a2, a3, a4, a5, a6 = symbols('a0:7')
alpha0, alpha1, alpha2, alpha3, alpha4, alpha5, alpha6 = symbols('alpha0:7')
q1, q2, q3, q4, q5, q6, q7 = symbols('q1:8')

theta1 = q1
theta2 = q2 - pi/2
theta3 = q3
theta4 = q4
theta5 = q5
theta6 = q6
theta7 = q7

s = {alpha0:     0, a0:   0   , d1: 0.75,
     alpha1: -pi/2, a1: 0.35  , d2: 0,
     alpha2:     0, a2: 1.25  , d3: 0,
     alpha3: -pi/2, a3: -0.054, d4: 1.5,
     alpha4:  pi/2, a4:   0   , d5: 0,
     alpha5: -pi/2, a5:   0   , d6: 0,
     alpha6:     0, a6:   0   , d7: 0.303, q7: 0}


In [608]:
def H_transform(alpha, a, d, theta):
    A = Matrix([[            cos(theta),           -sin(theta),           0,             a],
                [ sin(theta)*cos(alpha), cos(theta)*cos(alpha), -sin(alpha), -sin(alpha)*d],
                [ sin(theta)*sin(alpha), cos(theta)*sin(alpha),  cos(alpha),  cos(alpha)*d],
                [                 0,                 0,           0,             1]])
    return A
    

In [609]:
T0_1 = H_transform(alpha0, a0, d1, theta1)
T1_2 = H_transform(alpha1, a1, d2, theta2)
T2_3 = H_transform(alpha2, a2, d3, theta3)
T3_4 = H_transform(alpha3, a3, d4, theta4)
T4_5 = H_transform(alpha4, a4, d5, theta5)
T5_6 = H_transform(alpha5, a5, d6, theta6)
T6_EE = H_transform(alpha6, a6, d7, theta7)


print(T6_EE[0:3,3].evalf(subs=s))

Matrix([[0], [0], [0.303000000000000]])


In [612]:
T0_1 = T0_1.subs(s)
T1_2 = T1_2.subs(s)
T2_3 = T2_3.subs(s)
T3_4 = T3_4.subs(s)
T4_5 = T4_5.subs(s)
T5_6 = T5_6.subs(s)
T6_EE = T6_EE.subs(s)
T0_6 = simplify(T0_1*T1_2*T2_3*T3_4*T4_5*T5_6)
T0_EE = simplify(T0_1*T1_2*T2_3*T3_4*T4_5*T5_6*T6_EE)
T0_3 = simplify(T0_1*T1_2*T2_3)
T3_6 = simplify(T3_4*T4_5*T5_6)
R3_6 = T3_6 [:3, :3]
R0_3 = T0_3 [:3, :3]
R0_3T = T0_3[:3, :3].transpose()

print (T0_EE[0:3,3].evalf(subs={q1:-0.65, q2:0.45, q3:-0.36, q4:0.95, q5:0.79, q6:0.49}))


print ("R0_3=",R0_3)
print ("R0_3T=",R0_3T) 


Matrix([[2.16298054661547], [-1.42438431475958], [1.54309861553709]])
R0_3= Matrix([[sin(q2 + q3)*cos(q1), cos(q1)*cos(q2 + q3), -sin(q1)], [sin(q1)*sin(q2 + q3), sin(q1)*cos(q2 + q3), cos(q1)], [cos(q2 + q3), -sin(q2 + q3), 0]])
R0_3T= Matrix([[sin(q2 + q3)*cos(q1), sin(q1)*sin(q2 + q3), cos(q2 + q3)], [cos(q1)*cos(q2 + q3), sin(q1)*cos(q2 + q3), -sin(q2 + q3)], [-sin(q1), cos(q1), 0]])


In [613]:
def R_Rotation(roll, pitch, yaw):
    R_x = Matrix([[1,         0,          0],
                  [0, cos(roll), -sin(roll)],
                  [0, sin(roll),  cos(roll)]])

    R_y = Matrix([[ cos(pitch), 0, sin(pitch)],
                  [          0, 1,          0],
                  [-sin(pitch), 0, cos(pitch)]])

    R_z = Matrix([[cos(yaw),-sin(yaw), 0],
                  [sin(yaw), cos(yaw), 0],
                  [       0,        0, 1]])

    return R_z*R_y*R_x

def R_correction(roll, pitch, yaw):
    R_x = Matrix([[1,         0,          0],
                  [0, cos(roll), -sin(roll)],
                  [0, sin(roll),  cos(roll)]])

    R_y = Matrix([[ cos(pitch), 0, sin(pitch)],
                  [          0, 1,          0],
                  [-sin(pitch), 0, cos(pitch)]])

    R_z = Matrix([[cos(yaw),-sin(yaw), 0],
                  [sin(yaw), cos(yaw), 0],
                  [       0,        0, 1]])

    return R_z*R_y

In [614]:
 # from the test cases
    
px = 2.16135
py = -1.42635
pz = 1.55109

roll = 0.708611
pitch = 0.186356
yaw = -0.157931


In [615]:
R0_rpy = R_Rotation(roll, pitch, yaw) # end effector rotation matrix in URDF frame

R_corr = R_correction(0,-pi/2, pi) # rotation around y and z

R0_EE = R0_rpy*R_corr # corrected orientation matrix of the end effector matrix
            
wx = px - 0.303*R0_EE[0,2] 
wy = py - 0.303*R0_EE[1,2]
wz = pz - 0.303*R0_EE[2,2]



print("R0_EE=",R0_EE)
print("wx=", wx)
print("wy=", wy)
print("wz=", wz)




R0_EE= Matrix([[0.0365739909998583, -0.238489249459210, 0.970456192249153], [-0.664805602129873, -0.730853695260488, -0.154552216098079], [0.746120536213204, -0.639512121868283, -0.185279225560902]])
wx= 1.86730177374851
wy= -1.37952067852228
wz= 1.60722960534495


In [616]:


x = sqrt(wx**2 + wy**2)
w = atan2(wz - d1, x-a1)
b = sqrt((wz -d1)**2 + (x-a1)**2)
c = sqrt(d4**2 + 0.054**2)
A = acos((b**2 + a2**2 - c**2)/(2*b*a2))       
B = acos((a2**2 + c**2 - b**2)/(2*a2*c))

theta1 = atan2(wy, wx)# q1
theta1 = theta1.evalf(subs=s)
theta2 = pi/2 - w - A # q2
theta2 = theta2.evalf(subs=s)
theta3 = pi/2 - B + atan2(a3, d4) # q3
theta3 = theta3.evalf(subs=s)

T0_3 = T0_3.evalf(subs={q1: theta1, q2: theta2, q3: theta3})
R3_6 = T0_3[:3, :3].inv("LU")* R0_EE

#R3_6 = inv(array(R0_3, dtype='float')) * R0_EE

print("q1=", theta1)
print("q2=", theta2)
print("q3=", theta3)

print(T0_3[:3, :3])



q1= -0.636279983937866
q2= 0.410401660065528
q3= -0.252653773154244
Matrix([[0.126352922238237, 0.794325108297270, 0.594207506995989], [-0.0933467056422568, -0.586829578256010, 0.804311779492015], [0.987583582076776, -0.157094456975426, 0]])


In [617]:
r12, r13 = R3_6 [0,1], R3_6 [0,2]
r21, r22, r23 = R3_6 [1,0], R3_6 [1,1], R3_6 [1,2] 
r32, r33 = R3_6 [2,1], R3_6 [2,2]

theta4 = atan2(r33, -r13) # q4
theta4 = theta4.evalf()

theta5 = atan2(sqrt(r13**2 + r33**2),r23) # q5
theta5 = theta5.evalf()

theta6 = atan2(-r22, +r21) # q6
theta6 = theta6.evalf()

test_case = [[[2.16135,-1.42635,1.55109],
                  [0.708611,0.186356,-0.157931,0.661967]],
                  [1.89451,-1.44302,1.69366],
                  [-0.65,0.45,-0.36,0.95,0.79,0.49]]

t_1_e = abs(theta1-test_case[2][0])
t_2_e = abs(theta2-test_case[2][1])
t_3_e = abs(theta3-test_case[2][2])
t_4_e = abs(theta4-test_case[2][3])
t_5_e = abs(theta5-test_case[2][4])
t_6_e = abs(theta6-test_case[2][5])
    
print("q4=", theta4)
print("q5=", theta5)
print("q6=", theta6)

print ("\nTheta 1 error is: %04.8f" % t_1_e)
print ("Theta 2 error is: %04.8f" % t_2_e)
print ("Theta 3 error is: %04.8f" % t_3_e)
print ("Theta 4 error is: %04.8f" % t_4_e)
print ("Theta 5 error is: %04.8f" % t_5_e)
print ("Theta 6 error is: %04.8f" % t_6_e)


q4= 1.46960145630108
q5= 0.472001895809933
q6= -0.844444132991418

Theta 1 error is: 0.01372002
Theta 2 error is: 0.03959834
Theta 3 error is: 0.10734623
Theta 4 error is: 0.51960146
Theta 5 error is: 0.31799810
Theta 6 error is: 1.33444413


In [619]:
state = {q1:theta1, q2:theta2, q3: theta3, 
         q4:theta4, q5:theta5, q6: theta6}

T_total_pos = T0_EE[0:3,3]
T_total_or = T0_EE[:3,:3] 

print("T_total = ", [T_total_or.evalf(subs=state)],
                     [T_total_pos.evalf(subs=state)])


T_total =  [Matrix([
[0.0365739909998575, -0.238489249459209,  0.970456192249153],
[-0.664805602129874, -0.730853695260487, -0.154552216098079],
[ 0.746120536213203, -0.639512121868284, -0.185279225560901]])] [Matrix([
[ 2.16135],
[-1.42635],
[ 1.55109]])]


In [620]:
T_total_pos = T0_EE[0:3,3]
T_total_or = T0_EE[:3,:3] 

print("T_position = ", T_total_pos.evalf(subs=state))

print("T_orientation = ", T_total_or.evalf(subs=state))



T_position =  Matrix([[2.16135000000000], [-1.42635000000000], [1.55109000000000]])
T_orientation =  Matrix([[0.0365739909998575, -0.238489249459209, 0.970456192249153], [-0.664805602129874, -0.730853695260487, -0.154552216098079], [0.746120536213203, -0.639512121868284, -0.185279225560901]])


In [624]:
test_case = [[[2.16135,-1.42635,1.55109],
                  [0.708611,0.186356,-0.157931,0.661967]],
                  [1.89451,-1.44302,1.69366],
                  [-0.65,0.45,-0.36,0.95,0.79,0.49]]

T0_EE = simplify(T0_1*T1_2*T2_3*T3_4*T4_5*T5_6*T6_EE)
              
wc_x = abs(wx-test_case[1][0])
wc_y = abs(wy-test_case[1][1])
wc_z = abs(wz-test_case[1][2])
wc_offset = sqrt(wc_x**2 + wc_y**2 + wc_z**2)

ee_x_e = abs(T0_EE[0, 3].evalf(subs={q1: theta1, q2: theta2, q3: theta3, q4: theta4, q5: theta5, q6: theta6})-test_case[0][0][0])
ee_y_e = abs(T0_EE[1, 3].evalf(subs={q1: theta1, q2: theta2, q3: theta3, q4: theta4, q5: theta5, q6: theta6})-test_case[0][0][1])
ee_z_e = abs(T0_EE[2, 3].evalf(subs={q1: theta1, q2: theta2, q3: theta3, q4: theta4, q5: theta5, q6: theta6})-test_case[0][0][2])
ee_offset = sqrt(ee_x_e**2 + ee_y_e**2 + ee_z_e**2)
              
print("wc_error=",wc_offset) 

print("ee_error=",ee_offset)  

print("T0_EE=",T0_EE.evalf(subs={q1: theta1, q2: theta2, q3: theta3, q4: theta4, q5: theta5, q6: theta6}))



wc_error= 0.110646574841294
ee_error= 2.22044604925031e-16
T0_EE= Matrix([[0.0365739909998575, -0.238489249459209, 0.970456192249153, 2.16135000000000], [-0.664805602129874, -0.730853695260487, -0.154552216098079, -1.42635000000000], [0.746120536213203, -0.639512121868284, -0.185279225560901, 1.55109000000000], [0, 0, 0, 1.00000000000000]])


In [626]:
P_EE = Matrix([[2.16298054661547], [-1.42438431475958], [1.54309861553709]])

P_FK = Matrix([[2.1614], [-1.4264], [1.5511]])

error = P_FK-P_EE
error.norm()
         
print ("error=",error.norm())

error= 0.00840138488080479
